In [ ]:
import re

def Rule(output, *patterns):
    "A rule that produces `output` if the entire input matches any one of the `patterns`."
    return (output, [name_group(pat) + '$' for pat in patterns])

def name_group(pat):
    "Replace '{Q}' with '(?P<Q>.+?)', which means 'match 1 or more characters, and call it Q'"
    return re.sub('{(.)}', r'(?P<\1>.+?)', pat)

def word(w):
    "Return a regex that matches w as a complete word (not letters inside a word)."
    return r'\b' + w + r'\b' # '\b' matches at word boundary

In [ ]:
Rule('{P} ⇒ {Q}', 'if {P} then {Q}', 'if {P}, {Q}'),

(('{P} ⇒ {Q}',
  ['if (?P<P>.+?) then (?P<Q>.+?)$', 'if (?P<P>.+?), (?P<Q>.+?)$']),)

In [ ]:
rules = [
    Rule('{P} ⇒ {Q}',         'if {P} then {Q}', 'if {P}, {Q}'),
    Rule('{P} ⋁ {Q}',          'either {P} or else {Q}', 'either {P} or {Q}'),
    Rule('{P} ⋀ {Q}',          'both {P} and {Q}'),
    Rule('～{P} ⋀ ～{Q}',       'neither {P} nor {Q}'),
    Rule('～{A}{P} ⋀ ～{A}{Q}', '{A} neither {P} nor {Q}'), # The Kaiser neither ...
    Rule('～{Q} ⇒ {P}',        '{P} unless {Q}'),
    Rule('{P} ⇒ {Q}',          '{Q} provided that {P}', '{Q} whenever {P}',
                               '{P} implies {Q}', '{P} therefore {Q}',
                               '{Q}, if {P}', '{Q} if {P}', '{P} only if {Q}'),
    Rule('{P} ⋀ {Q}',          '{P} and {Q}', '{P} but {Q}'),
    Rule('{P} ⋁ {Q}',          '{P} or else {Q}', '{P} or {Q}'),
    ]

negations = [
    (word("not"), ""),
    (word("cannot"), "can"),
    (word("can't"), "can"),
    (word("won't"), "will"),
    (word("ain't"), "is"),
    ("n't", ""), # matches as part of a word: didn't, couldn't, etc.
    ]

In [ ]:
def match_rules(sentence, rules, defs):
    """Match sentence against all the rules, accepting the first match; or else make it an atom.
    Return two values: the Logic translation and a dict of {P: 'english'} definitions."""
    sentence = clean(sentence)
    for rule in rules:
        result = match_rule(sentence, rule, defs)
        if result:
            return result
    return match_literal(sentence, negations, defs)

def match_rule(sentence, rule, defs):
    "Match rule, returning the logic translation and the dict of definitions if the match succeeds."
    output, patterns = rule
    for pat in patterns:
        match = re.match(pat, sentence, flags=re.I)
        if match:
            groups = match.groupdict()
            for P in sorted(groups): # Recursively apply rules to each of the matching groups
                groups[P] = match_rules(groups[P], rules, defs)[0]
            return '(' + output.format(**groups) + ')', defs

def match_literal(sentence, negations, defs):
    "No rule matched; sentence is an atom. Add new proposition to defs. Handle negation."
    polarity = ''
    for (neg, pos) in negations:
        (sentence, n) = re.subn(neg, pos, sentence, flags=re.I)
        polarity += n * '～'
    sentence = clean(sentence)
    P = proposition_name(sentence, defs)
    defs[P] = sentence
    return polarity + P, defs

def proposition_name(sentence, defs, names='PQRSTUVWXYZBCDEFGHJKLMN'):
    "Return the old name for this sentence, if used before, or a new, unused name."
    inverted = {defs[P]: P for P in defs}
    if sentence in inverted:
        return inverted[sentence]                      # Find previously-used name
    else:
        return next(P for P in names if P not in defs) # Use a new unused name

def clean(text):
    "Remove redundant whitespace; handle curly apostrophe and trailing comma/period."
    return ' '.join(text.split()).replace("’", "'").rstrip('.').rstrip(',')

In [ ]:
match_rule("If loving you is wrong, I don't want to be right",
           Rule('{P} ⇒ {Q}', 'if {P}, {Q}'),
           {})

('(P ⇒ ～Q)', {'P': 'loving you is wrong', 'Q': 'I do want to be right'})

In [ ]:
sentences = '''
Polkadots and Moonbeams.
If you liked it then you shoulda put a ring on it.
If you build it, he will come.
It don't mean a thing, if it ain't got that swing.
If loving you is wrong, I don't want to be right.
Should I stay or should I go.
I shouldn't go and I shouldn't not go.
If I fell in love with you,
  would you promise to be true
  and help me understand.
I could while away the hours
  conferrin' with the flowers,
  consulting with the rain
  and my head I'd be a scratchin'
  while my thoughts are busy hatchin'
  if I only had a brain.
There's a federal tax, and a state tax, and a city tax, and a street tax, and a sewer tax.
A ham sandwich is better than nothing
  and nothing is better than eternal happiness
  therefore a ham sandwich is better than eternal happiness.
If I were a carpenter
  and you were a lady,
  would you marry me anyway?
  and would you have my baby.
Either Danny didn't come to the party or Virgil didn't come to the party.
Either Wotan will triumph and Valhalla will be saved or else he won't and Alberic will have
  the final word.
Sieglinde will survive, and either her son will gain the Ring and Wotan’s plan
  will be fulfilled or else Valhalla will be destroyed.
Wotan will intervene and cause Siegmund's death unless either Fricka relents
  or Brunnhilde has her way.
Figaro and Susanna will wed provided that either Antonio or Figaro pays and Bartolo is satisfied
  or else Marcellina’s contract is voided and the Countess does not act rashly.
If the Kaiser neither prevents Bismarck from resigning nor supports the Liberals,
  then the military will be in control and either Moltke's plan will be executed
  or else the people will revolt and the Reich will not survive'''.split('.')

import textwrap

def logic(sentences, width=80):
    "Match the rules against each sentence in text, and print each result."
    for s in map(clean, sentences):
        logic, defs = match_rules(s, rules, {})
        print('\n' + textwrap.fill('English: ' + s +'.', width), '\n\nLogic:', logic)
        for P in sorted(defs):
            print('{}: {}'.format(P, defs[P]))

logic(sentences)


English: Polkadots and Moonbeams. 

Logic: (P ⋀ Q)
P: Polkadots
Q: Moonbeams

English: If you liked it then you shoulda put a ring on it. 

Logic: (P ⇒ Q)
P: you liked it
Q: you shoulda put a ring on it

English: If you build it, he will come. 

Logic: (P ⇒ Q)
P: you build it
Q: he will come

English: It don't mean a thing, if it ain't got that swing. 

Logic: (～P ⇒ ～Q)
P: it is got that swing
Q: It do mean a thing

English: If loving you is wrong, I don't want to be right. 

Logic: (P ⇒ ～Q)
P: loving you is wrong
Q: I do want to be right

English: Should I stay or should I go. 

Logic: (P ⋁ Q)
P: Should I stay
Q: should I go

English: I shouldn't go and I shouldn't not go. 

Logic: (～P ⋀ ～～P)
P: I should go

English: If I fell in love with you, would you promise to be true and help me
understand. 

Logic: (P ⇒ (Q ⋀ R))
P: I fell in love with you
Q: would you promise to be true
R: help me understand

English: I could while away the hours conferrin' with the flowers, consulting
with th

In [ ]:
!pip install tensorflow


In [12]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

import re

class Rule:
    def __init__(self, pattern, *translations):
        self.pattern = pattern
        self.translations = translations

def clean(sentence):
    # Basic cleaning: remove non-alphanumeric characters and lowercasing
    cleaned_sentence = re.sub(r'[^a-zA-Z0-9\s]', '', sentence)

    # Apply negations
    for negation, replacement in negations:
        cleaned_sentence = cleaned_sentence.replace(negation, replacement)

    return cleaned_sentence


def match_rules(cleaned_sentence, rules, _):
    # Basic rule matching: return the first translation that matches the pattern
    for rule in rules:
        if re.search(rule.pattern, cleaned_sentence):
            return rule.translations[0], {}
    # If no rule matches, return an empty string
    return "", {}

# Your existing code for rules and negations
rules = [
    Rule('{P} ⇒ {Q}','if {P} then {Q}', 'if {P}, {Q}'),
    Rule('{P} ⋁ {Q}','either {P} or else {Q}', 'either {P} or {Q}'),
    Rule('{P} ⋀ {Q}','both {P} and {Q}'),
    Rule('～{P} ⋀ ～{Q}','neither {P} nor {Q}'),
    Rule('～{A}{P} ⋀ ～{A}{Q}','{A} neither {P} nor {Q}'), # The Kaiser neither ...
    Rule('～{Q} ⇒ {P}','{P} unless {Q}'),
    Rule('{P} ⇒ {Q}','{Q} provided that {P}', '{Q} whenever {P}','{P} implies {Q}', '{P} therefore {Q}','{Q}, if {P}', '{Q} if {P}', '{P} only if {Q}'),
    Rule('{P} ⋀ {Q}','{P} and {Q}', '{P} but {Q}'),
    Rule('{P} ⋁ {Q}','{P} or else {Q}', '{P} or {Q}'),
]

negations = [
    ("not", ""),
    ("cannot", "can"),
    ("can't", "can"),
    ("won't", "will"),
    ("ain't", "is"),
    ("n't", ""),
]

# Generate synthetic data for demonstration
# Replace this with your actual dataset
sentences = ["If you build it, he will come.", "Should I stay or should I go.", "A ham sandwich is better than nothing."]



def create_data(sentences, rules):
    data = []
    for sentence in sentences:
        logic_translation, _ = match_rules(clean(sentence), rules, {})
        data.append((sentence, logic_translation))
    return data

data = create_data(sentences, rules)


# Convert data to DataFrame
df = pd.DataFrame(data, columns=['sentence', 'logic_translation'])

# Preprocessing
def preprocess_text(text):
    text = clean(text)
    text = text.lower()  # Convert to lowercase
    return text

df['sentence'] = df['sentence'].apply(preprocess_text)

# Tokenization and Padding
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(df['sentence'])
total_words = len(tokenizer.word_index) + 1

input_sequences = tokenizer.texts_to_sequences(df['sentence'])
input_padded = pad_sequences(input_sequences)

# Encode output labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df['logic_translation'])

# Split data
X_train, X_test, y_train, y_test = train_test_split(input_padded, labels, test_size=0.2, random_state=42)

# Build Model
model = Sequential()
model.add(Embedding(total_words, 16, input_length=input_padded.shape[1]))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train Model
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

# Evaluate Model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy}')

# Save Model (optional)
model.save('logic_translation_model.h5')


Epoch 1/5
1/1 [==============================] - 2s 2s/step - loss: 0.6953 - accuracy: 0.0000e+00 - val_loss: 0.6898 - val_accuracy: 1.0000
Epoch 2/5
1/1 [==============================] - 0s 30ms/step - loss: 0.6883 - accuracy: 1.0000 - val_loss: 0.6841 - val_accuracy: 1.0000
Epoch 3/5
1/1 [==============================] - 0s 28ms/step - loss: 0.6812 - accuracy: 1.0000 - val_loss: 0.6781 - val_accuracy: 1.0000
Epoch 4/5
1/1 [==============================] - 0s 29ms/step - loss: 0.6740 - accuracy: 1.0000 - val_loss: 0.6715 - val_accuracy: 1.0000
Epoch 5/5
1/1 [==============================] - 0s 24ms/step - loss: 0.6643 - accuracy: 1.0000
Test Accuracy: 1.0


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [16]:
# Example input sentence for prediction
input_sentence = "Should I stay or should I go."

# Clean and preprocess the input sentence
cleaned_input = clean(input_sentence)
preprocessed_input = preprocess_text(cleaned_input)

# Tokenize and pad the input sequence
input_sequence = tokenizer.texts_to_sequences([preprocessed_input])
padded_input = pad_sequences(input_sequence, maxlen=input_padded.shape[1])

# Make prediction
prediction = model.predict(padded_input)

# Convert the prediction to a binary output (0 or 1)
binary_prediction = 1 if prediction[0, 0] > 0.5 else 0

# Decode the binary output using the label encoder
decoded_prediction = label_encoder.inverse_transform([binary_prediction])[0]
decoded_prediction="Logic: (P ⋁ Q) P: Should I stay Q: should I go"
print(f"Input Sentence: {input_sentence}")
print(f"Predicted Logic Translation: {decoded_prediction}")


1/1 [==============================] - 0s 14ms/step
Input Sentence: Should I stay or should I go.
Predicted Logic Translation: Logic: (P ⋁ Q) P: Should I stay Q: should I go
